In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_colleges = pd.read_csv("../csv_clean/colleges.csv")
df_lycees = pd.read_csv("../csv_clean/lycees.csv")
df_transport = pd.read_csv("../csv_clean/transports.csv")
pd.set_option('display.precision', 15)

df_transport


,insee,commune,lat,lon
0,69384.0,Lyon 4e Arrondissement,45.779495923094579,4.837352539879999
1,69273.0,Corbas,45.682868781870653,4.909324275377001
2,69384.0,Lyon 4e Arrondissement,45.779963769712509,4.834530774781000
3,69384.0,Lyon 4e Arrondissement,45.777121008500849,4.836884357859000
4,69384.0,Lyon 4e Arrondissement,45.777198018673829,4.833544046147000
...,...,...,...,...
5009,69387.0,Lyon 7e Arrondissement,45.722498525760692,4.845405158745000
5010,69387.0,Lyon 7e Arrondissement,45.728527053853078,4.845644982505000
5011,69387.0,Lyon 7e Arrondissement,45.728476062013087,4.845651012156000
5012,69199.0,Saint-Fons,45.709496077232018,4.860892958133999


In [3]:
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display

# --- Liste des communes disponibles (avec leurs codes INSEE) ---
communes_insee = pd.concat([
    df_colleges[['commune', 'insee']],
    df_lycees[['commune', 'insee']],
    df_transport[['commune', 'insee']] if 'commune' in df_transport.columns and 'insee' in df_transport.columns else pd.DataFrame(columns=['commune', 'insee'])
]).drop_duplicates().sort_values('commune')

options_communes = ["Toutes"] + list(communes_insee['commune'])

# --- Widgets interactifs ---
dropdown_commune = widgets.Dropdown(
    options=options_communes,
    description="Commune :",
    value="Toutes"
)

checkbox_colleges = widgets.Checkbox(
    value=True,
    description="Collèges (rouge)"
)

checkbox_lycees = widgets.Checkbox(
    value=True,
    description="Lycées (bleu)"
)

checkbox_transport = widgets.Checkbox(
    value=False,
    description="Transports (vert)"
)

# --- Fonction pour mettre à jour la carte ---
def afficher_carte(commune, afficher_colleges, afficher_lycees, afficher_transports):
    # Filtrage des établissements selon la commune / code INSEE
    if commune == "Toutes":
        cols = df_colleges
        lycees = df_lycees
        transports = df_transport
    else:
        insee_code = communes_insee[communes_insee['commune'] == commune]['insee'].values[0]
        cols = df_colleges[df_colleges["insee"] == insee_code]
        lycees = df_lycees[df_lycees["insee"] == insee_code]
        transports = df_transport[df_transport["insee"] == insee_code] if "insee" in df_transport.columns else df_transport

    # Préparation des datasets à afficher
    etablissements = []
    if afficher_colleges:
        etablissements.append(("red", "Collège", cols))
    if afficher_lycees:
        etablissements.append(("blue", "Lycée", lycees))

    # Vérification qu'au moins une catégorie est cochée
    if not etablissements and not afficher_transports:
        display(widgets.HTML("<b style='color:red;'>Coche au moins un type d'établissement ou transport.</b>"))
        return

    # Calcul du centre de la carte
    all_points = pd.concat([df for _, _, df in etablissements if not df.empty])
    if afficher_transports and not transports.empty:
        all_points = pd.concat([all_points, transports]) if not all_points.empty else transports
    if len(all_points) > 0:
        center_lat = all_points["lat"].mean()
        center_lon = all_points["lon"].mean()
    else:
        center_lat, center_lon = 45.75, 4.85  # coordonnées par défaut (Lyon)

    # Création de la carte
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Ajout des marqueurs pour collèges et lycées
    for color, label, df in etablissements:
        for _, row in df.iterrows():
            folium.CircleMarker(
                location=[row["lat"], row["lon"]],
                radius=6,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.8,
                popup=f"{'🏫' if label=='Collège' else '🎓'} {row['nom']} ({label})<br>Commune : {row['commune']}"
            ).add_to(m)

    # Ajout des transports si coché
    if afficher_transports:
        for _, row in transports.iterrows():
            folium.CircleMarker(
                location=[row["lat"], row["lon"]],
                radius=6,
                color="green",
                fill=True,
                fill_color="green",
                fill_opacity=0.8,
                popup=row.get("nom", "Transport")
            ).add_to(m)

    # Affichage de la carte
    display(m)

# --- Lier les widgets à la fonction via interactive_output ---
out = widgets.interactive_output(
    afficher_carte,
    {
        "commune": dropdown_commune,
        "afficher_colleges": checkbox_colleges,
        "afficher_lycees": checkbox_lycees,
        "afficher_transports": checkbox_transport
    }
)

# --- Affichage de l'interface ---
ui = widgets.VBox([dropdown_commune, widgets.HBox([checkbox_colleges, checkbox_lycees, checkbox_transport])])
display(ui, out)


Output()